## Imports
`osgeo` requires `gdal`

In [1]:
import os, csv
#from osgeo import gdal
import fiona
import pandas as pd
import numpy as np
import geopandas as gpd
import shapely
from shapely import wkt
import matplotlib.pyplot as plt
from SPARQLWrapper import SPARQLWrapper, CSV
import folium
from folium.plugins import MarkerCluster

## Configuration

In [2]:
# Configuring SPARQL Wrapper
sparql = SPARQLWrapper("https://stko-kwg.geog.ucsb.edu/graphdb/repositories/KWG")
sparql.setReturnFormat(CSV)

# Configuring this DUCK
output_dir = "./output"

## Library Functions

In [3]:
def save_query(result, filename, output_dir=output_dir, mode="wb"):
    """Given a SPARQLWrapper result and a target filename, output the results to `output_dir` according to specified mode. Default output_dir is set in Configuration Cell Default mode is `write binary`."""
    output_path = os.path.join(output_dir, filename)
    with open(output_path, mode) as file:
        file.write(result)

def query_kwg(query, endpoint=sparql, save_file = False, filename = None):
    """Given a configured endpoint, accept query and execute it against the endpoint. Endpoint is bydefault set in the configuration cell."""
    endpoint.setQuery(query)
    res = None
    try:
        res = endpoint.query().convert()
    except Exception as e:
        print(f"There was a problem retrieving the fires: {e}")

    if save_file and res != None:
        save_query(res, filename)

## Query Examples
text

### Wildfire Query
This query will return all fires and their context in North America.

In [4]:
fire_query = """PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX kwg-ont: <http://stko-kwg.geog.ucsb.edu/lod/ontology/>
PREFIX kwgr: <http://stko-kwg.geog.ucsb.edu/lod/resource/>
PREFIX time: <http://www.w3.org/2006/time#>
PREFIX geo: <http://www.opengis.net/ont/geosparql#>
select * where { 
    ?fire rdf:type kwg-ont:MTBSFire .
    ?fire kwg-ont:sfWithin kwgr:Earth.North_America.United_States.USA.5.42_1 .
    ?fire kwg-ont:hasTemporalScope ?temporal_scope .
    ?fire kwg-ont:hasFireName ?name .
    ?fire geo:hasGeometry ?geom .
    ?geom geo:asWKT ?wkt .
    ?temporal_scope time:inXSDgYear ?year .	
}"""
query_kwg(fire_query, save_file = True, filename = "fires.csv")

### County Query
This query will return all counties. These are represented as AdminRegion Level 3 that are within a state which are in the United States. We return name, geometries, and associated URIs.

In [5]:
county_query = """PREFIX gnis: <http://gnis-ld.org/lod/gnis/ontology/>
PREFIX kwg-ont: <http://stko-kwg.geog.ucsb.edu/lod/ontology/>
PREFIX kwgr: <http://stko-kwg.geog.ucsb.edu/lod/resource/>

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX geo: <http://www.opengis.net/ont/geosparql#>
select * where { 
    ?county a kwg-ont:AdministrativeRegion_3 ;
    		kwg-ont:hasFIPS ?fips ;
           	geo:hasGeometry ?geometry ;
      		kwg-ont:sfWithin ?state ;
           	rdfs:label ?name .
    ?state a kwg-ont:AdministrativeRegion_2 ;
           kwg-ont:sfWithin kwgr:Earth.North_America.United_States.USA .
    ?geometry geo:asWKT ?wkt .
}
"""
query_kwg(county_query, save_file = True, filename = "counties.csv")

### State Query
This query will return all States. These are represented as AdminRegion Level 2 that are within the United States. We return name, geometries, and associated URIs.

In [6]:
state_query = """PREFIX gnis: <http://gnis-ld.org/lod/gnis/ontology/>
PREFIX kwg-ont: <http://stko-kwg.geog.ucsb.edu/lod/ontology/>
PREFIX kwgr: <http://stko-kwg.geog.ucsb.edu/lod/resource/>

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX geo: <http://www.opengis.net/ont/geosparql#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
select ?name ?state ?fips ?wkt where { 
    ?state a kwg-ont:AdministrativeRegion_2 ;
           kwg-ont:sfWithin kwgr:Earth.North_America.United_States.USA ;
           kwg-ont:hasFIPS ?fips ;
           geo:hasGeometry ?geometry ;
           rdfs:label ?name .
    ?geometry geo:asWKT ?wkt .
}
"""
query_kwg(state_query, save_file = True, filename = "states.csv")

### City Query
This query is currently not possible for cities within the United States. Certain cities can be found via their eponymous counties, however (e.g., Santa Barbara).

In [7]:
city_query = """ NOT POSSIBLE """
# query_kwg(city_query, save_file = True, filename = "cities.csv")

### ZIP Code Areas
This query will return URIs for all zip codes (which are necessarily within the USA), their label (i.e., their five digit number), URIs to their encapsulating geometry nodes, and the wkt representation of their geometry.

In [9]:
zip_query = """PREFIX gnis: <http://gnis-ld.org/lod/gnis/ontology/>
PREFIX kwg-ont: <http://stko-kwg.geog.ucsb.edu/lod/ontology/>
PREFIX kwgr: <http://stko-kwg.geog.ucsb.edu/lod/resource/>

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX geo: <http://www.opengis.net/ont/geosparql#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>   
select * where {
?city a kwg-ont:ZipCodeArea ;
          rdfs:label ?label ;
          geo:hasGeometry ?geometry .
    ?geometry geo:asWKT ?wkt .
}
"""
query_kwg(zip_query, save_file = True, filename = "zip_codes.csv")

## Visualizations